<a href="https://colab.research.google.com/github/Lior-Baruch/SFT_llama_qlora/blob/working_branch/FineTune_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install all dependencies
!pip install -q -U peft transformers datasets bitsandbytes trl accelerate wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
from huggingface_hub import notebook_login

# log in to the Hugging Face hub (required for private datasets/models)
notebook_login()

load base model llama-2-7b and tokenizer

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig

# Load the 7b llama-2 model
model_id = "meta-llama/Llama-2-7b-hf"

# Set quantization config (to save memory)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

# Load model, quantized
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# Set it to a new token to correctly attend to EOS tokens.
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

add LoRA adapter

In [ ]:
# Define LoRA ("low-rank attention") config
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

# Add LoRA adapter to model
model.add_adapter(lora_config)

load chat dataset

In [ ]:
from datasets import load_dataset

# Load dataset ultrachat (User Assisted Chat Dataset)
train_dataset = load_dataset("stingning/ultrachat", split="train[:1%]")

define training args

In [ ]:
from transformers import TrainingArguments

# Define training arguments
YOUR_HF_USERNAME = "LBK95" # Set my HF username

output_dir = f"{YOUR_HF_USERNAME}/llama-7b-qlora-ultrachat" # Set output directory for fine-tuned model
per_device_train_batch_size = 4  # Set batch size per device for training
gradient_accumulation_steps = 4  # Set the number of gradient accumulation steps
optim = "paged_adamw_32bit" # Set optimizer
save_steps = 10 # Set the number of steps before saving
logging_steps = 10 # Set the number of steps before logging
learning_rate = 2e-4 # Set learning rate
max_grad_norm = 0.3 # Set max gradient norm
max_steps = 200 # Set max steps
warmup_ratio = 0.03 # Set warmup ratio for learning rate scheduler
lr_scheduler_type = "constant" # Set learning rate scheduler type


# Create TrainingArguments object with all of our defined arguments
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
    push_to_hub=True,
)


define formatting_func

In [ ]:
from trl import SFTTrainer

# formatting function for the SFTTrainer (to format the dataset correctly)
def formatting_func(example):
    text = f"### USER: {example['data'][0]}\n### ASSISTANT: {example['data'][1]}"
    return text

create SFTTrainer

In [ ]:
# Create SFTTrainer object (it is a wrapper around the Trainer object for Supervised-Fine-Tuning)
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    packing=True,
    dataset_text_field="id",
    tokenizer=tokenizer,
    max_seq_length=1024,
    formatting_func=formatting_func,
)

train

In [ ]:
# Train the model using the SFTTrainer (this will take a while, and save the model to the output directory)
trainer.train()

test the base model and fine-tuned model

In [ ]:
text = "### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?### Assistant:"

inputs = tokenizer(text, return_tensors="pt").to(0)
model.enable_adapters()
outputs = model.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)
print()
print("After attaching Lora adapters:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

In [ ]:
model.disable_adapters()
outputs = model.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

print("Before Lora:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

load our saved model from the hub

In [6]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

model_id = f"LBK95/llama-7b-qlora-ultrachat"

tokenizer = AutoTokenizer.from_pretrained(model_id)

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

model_loaded = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config
)



tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/80.0M [00:00<?, ?B/s]

In [7]:
print(model_loaded)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleD

test our base model vs fine-tuned model

In [13]:
text = "### USER: How to plan a picnic?### Assistant:"
inputs = tokenizer(text, return_tensors="pt").to(0)

# with adaptor (fine-tuned model)
model_loaded.enable_adapters()
outputs = model_loaded.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

print("After attaching Lora adapters:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))
print()
# without adaptor (base model)
model_loaded.disable_adapters()
outputs = model_loaded.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

print("Before attaching Lora adapters:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

After attaching Lora adapters:
<s> ### USER: How to plan a picnic?### Assistant: 1. ### Step 1: Decide on a picnic spot: Choose a picnic spot that's scenic and has a lot of greenery.

2. Plan the menu: Decide on the food you want to eat, and make sure it's easy to transport and store.

3. Pack the food: Pack the food in coolers or baskets, and make sure it's easy to transport.

4. Pack the essentials: Pack the essentials like napkins, plates, utensils, and a cooler with ice.

5. Pack the non-food items: Pack the non-food items like a blanket, a picnic table, and a portable speaker.

6. Plan the transportation: Decide on the transportation method, whether it's a car, a bike, or a public transportation.

7. Pack the entertainment: Pack the entertainment like a portable speaker, a board game, or a book.

8. Pack the sun protection: Pack the sun protection like sunscreen, a hat, and sunglasses.

9

Before attaching Lora adapters:
<s> ### USER: How to plan a picnic?### Assistant: What do yo